In [ ]:
%matplotlib notebook
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate
import h5py
from model.DEM import DeepEuler
from model.Euler import Euler
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"],
    "font.size": 12.0})

In [ ]:
##Load results from text files, if C++ solvers were used
#dem = np.loadtxt('simulations/lotka_dem.txt')
#euler = np.loadtxt('simulations/lotka_euler_0.1.txt')

In [ ]:
def lotka( t, x):
    y = np.empty(x.shape)
    y[0] =  x[0] - x[0]*x[1]
    y[1] = -x[1] + x[0]*x[1]
    return y

In [ ]:
sol = scipy.integrate.solve_ivp(lotka, [0, 50], [ 2.0, 1.0], rtol=1e-6, atol=1e-6)

In [ ]:
eul_sol = scipy.integrate.solve_ivp(lotka, [0, 50], [ 2.0, 1.0], method=Euler, h=0.1)

In [ ]:
dem_sol = scipy.integrate.solve_ivp(lotka, [0, 50], [ 2.0, 1.0], method=DeepEuler, h=0.1, absolute_time=True,
                                    traced_model_path="training/traced_model_lotka_final_e283_2110141659.pt")

In [ ]:
plt.figure(num="pic",figsize=(8,6))
plt.scatter(sol.y[0,:],sol.y[1,:], s=10, label="Dopri")
#plt.scatter(euler[:250,1],euler[:250,2],s=10, label="Euler")
plt.scatter(eul_sol.y[0,:350],eul_sol.y[1,:350], s=10, label="Euler")
plt.scatter(dem_sol.y[0,:350],dem_sol.y[1,:350], s=10, label="DEM")
#plt.scatter(dem[:,1],dem[:,2],s=10, label="DEM C++")
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.legend()
plt.show()

In [ ]:
plt.figure(num="Comparison")
plt.plot(sol.t,sol.y[0,:], color="black", label="Dopri")
#plt.plot(euler[:,0],euler[:,1], color="silver", label="Euler")
plt.plot(eul_sol.t,eul_sol.y[0,:], color="purple", label="Euler")
#plt.plot(dem[:,0],dem[:,1], linestyle="--", dashes=(5,5), color="cyan", label="DEM")
plt.plot(dem_sol.t,dem_sol.y[0,:], linestyle="--", dashes=(5,5), color="orange", label="DEM")
plt.xlabel("$t$")
plt.ylabel("$x_1$")
plt.xlim([0, 50])
plt.legend()
plt.show()

In [ ]:
t_eval = np.linspace(0,50, 501)
dopri_sol = scipy.integrate.solve_ivp(lotka, [0, 50], [ 2.0, 1.0], rtol=1e-6, atol=1e-6, t_eval=t_eval)
euler_residual = np.abs(eul_sol.y[0,:]-dopri_sol.y[0,:])
dem_residual = np.abs(dem_sol.y[0,:]-dopri_sol.y[0,:])
plt.figure(num="ErrorComparison")
plt.plot(t_eval, euler_residual, color="purple", label="Euler")
plt.plot(t_eval,dem_residual, linestyle="--", dashes=(5,5), color="orange", label="DEM")
plt.xlabel("$t$")
plt.ylabel("$\\left| x_1-x_{1,DP} \\right|$")
plt.xlim([0, 50])
plt.legend()
plt.show()

In [ ]:
eul_cumulative = np.cumsum(euler_residual)
dem_cumulative = np.cumsum(dem_residual)
plt.figure(num="CumulativeErrorComparison")
plt.plot(t_eval, eul_cumulative, color="purple", label="Euler")
plt.plot(t_eval,dem_cumulative, linestyle="--", dashes=(5,5), color="orange", label="DEM")
plt.xlabel("$t$")
plt.ylabel("$\\sum \\left| x_1-x_{1,DP} \\right|$")
plt.xlim([0, 50])
plt.legend()
plt.show()

In [ ]:
np.max(dem_cumulative)